In [1]:
using SymPy
using LinearAlgebra
using Plots
using StaticArrays
using Combinatorics
using TimerOutputs
using LowRankApprox
using SpecialFunctions

In [2]:
include("factor.jl")
include("util.jl")
include("gegenbauer.jl")
include("hyperspherical.jl")

hypospherical (generic function with 1 method)

In [8]:
dct_n=100
kern          = 1 / (1+r^2)
lkern         = lambdify(kern)
mat_kern(x,y) = 1 / (1+norm(x-y)^2)

d = 2
x_vecs = [randn(d)/8 for _ in 1:1000]
max_norm = max(maximum(norm.(x_vecs)), maximum(norm.(x_vecs)))
b = 2max_norm
println(b)
to = TimerOutput()

fkt_deg = 12
dover2 = convert(Int, fkt_deg/2)

cfg = fkt_config(fkt_deg, d, b, dct_n, to)
U_mat, V_mat = cheb_fkt(lkern, x_vecs, x_vecs, cfg)

truth_mat  = mat_kern.(x_vecs, permutedims(x_vecs))
svecs, svals = svd(truth_mat);
guess = U_mat*V_mat
error = norm(guess-truth_mat)/norm(truth_mat)
println("FKT Error: ", error)

0.9521729705208397
FKT Error: 0.0010324055586911057


# Generate trans table so we can generate orthogonal polynomials

In [ ]:
weight = r^(d-1)
polynomials = [Sym(1)]
polynomials[1] /= sqrt(integrate(weight*(polynomials[1]^2), (r,0,b)))
for i in 2:(fkt_deg+1)
    current = r^(i-1)
    for j in 1:(i-1)
        current -= polynomials[j] * (integrate(weight*polynomials[j]*r^(i-1), (r, 0, b))
                / integrate(weight*polynomials[j]^2, (r, 0, b)))
    end
    norm = sqrt(integrate(weight*current^2, (r,0,b)))
    push!(polynomials, expand(current/norm))
end
l_polys = []
for p in polynomials
    push!(l_polys, lambdify(p, [r]))
end

B = zeros((fkt_deg+1),(fkt_deg+1))
for i in 1:(fkt_deg+1)
    for j in 1:(fkt_deg+1)
        if j == 1
            B[i,j] = subs(polynomials[i], r=>0)
        else
            B[i,j] = polynomials[i].coeff(r^(j-1))
        end
    end
end
Binv = inv(B)

a_vals = zeros(fkt_deg+1) # kern's coefs in cheb poly basis
for i in 0:(fkt_deg)
    a_vals[i+1] = dct(lkern, i, b, dct_n)
end
pij = get_pij_table(fkt_deg+1)
transtable = zeros(Float64, fkt_deg+1, fkt_deg+1, fkt_deg+1)
for j in 0:dover2
    for new_i in 0:(fkt_deg-j)
        for n in j:(fkt_deg-max(new_i,j))
            if mod(j+n, 2) != 0
                continue
            end
            for m in max(new_i,j):(fkt_deg-n)
                if mod(m+j, 2) != 0
                    continue
                end
                for k3 in j:min(m,n)
                    if mod(k3+n, 2) != 0
                        continue
                    end
                    for i in (n+m):fkt_deg
                        m1 = convert(Int,((n-k3)/2))
                        m2 = convert(Int,((m-k3)/2))
                    transtable[j+1, new_i+1, n+1] += (pij[i+1, n+m+1] # here
                                        * a_vals[i+1]
                                        *Binv[m+1, new_i+1]
                                        * A(j, k3, 1//2)
                                        *(1-delta(0,i)/2) # here
                                        *multinomial([m1,m2,k3]...) #here
                                        *((1.0/b)^(n+m)) #here
                                        *((-2.0)^k3)) #here
                    end
                end
            end
        end
    end
end

In [ ]:
polynomials

In [ ]:
U_polys = Dict()
V_polys = Dict()

x = Sym("x")
# There is a u and v poly for every j, h, i
# For now we will ignore the harmonics and just inject the delta
for j in 0:dover2
    for new_i in 0:(fkt_deg - j)
        x_poly=0
        for n in j:2:(fkt_deg-max(new_i,j))
            x_poly += (x^n)*transtable[j+1, new_i+1, n+1]
        end
        U_polys[(j, new_i)] = x_poly
        V_polys[(j, new_i)] = subs(polynomials[new_i+1], r=>x)
    end
end

In [ ]:
integrations = Dict()
for j in 0:dover2
    N_k_alpha = gegenbauer_normalizer(d, j)
    for new_i in 0:(fkt_deg - j)
        for new_ip in 0:(fkt_deg - j)
            prod_poly =  U_polys[(j, new_i)]*V_polys[(j, new_ip)]
            integrations[(j, new_i, new_ip)] =  N_k_alpha*integrate(prod_poly*x^2, (x, 0,b))
        end
    end
end

In [ ]:
pole_count = 0
for j in 0:dover2
    for h in 1:length(get_multiindices(d, j))
        for new_i in 0:(fkt_deg - j)
            pole_count += 1
        end
    end
end

In [ ]:
Cmat = zeros(pole_count,pole_count)
pole_count = 0
integral_count=0
for j in 0:dover2
    for h in 1:length(get_multiindices(d, j))
        for new_i in 0:(fkt_deg - j)
            pole_count += 1
            pole_countp = 0
            for jp in 0:dover2
                for hp in 1:length(get_multiindices(d, jp))
                    for new_ip in 0:(fkt_deg - jp)
                       pole_countp += 1
                        if j != jp || h != hp 
                            continue
                        end
                        Cmat[pole_count, pole_countp] = integrations[(j, new_i, new_ip)]
                    end
                end
            end
        end
    end
end

In [ ]:
plt = plot()
all_evals = []
all_evecs = []
for smaller_deg in 4:2:12
    smaller_dover2 = convert(Int, smaller_deg/2)
    smaller_pole_count = 0
    for j in 0:smaller_dover2
        for h in 1:length(get_multiindices(d, j))
            for new_i in 0:(smaller_deg - j)
                smaller_pole_count += 1
            end
        end
    end
    evals, evecs = eigen(Cmat[1:smaller_pole_count, 1:smaller_pole_count]);
    push!(all_evals, evals)
    push!(all_evecs, evecs)
    eigenguess = transpose(V_mat)[:,1:size(evecs,1)]*evecs*diagm(evals)*conj(transpose(evecs))*conj(V_mat)[1:size(evecs,1),:]; 
    println(smaller_deg, " ", norm(eigenguess-truth_mat)/norm(truth_mat))
    
    revevals = max.(1e-6, sort(abs.(real(evals)))[end:-1:1])
    cap = min(60,length(revevals))
    plot!(1:cap, revevals[1:cap]/revevals[1], yscale=:log10)
end
plt

In [ ]:
# revevals = real(evals[end:-1:1])
evals = all_evals[end]
revevals = max.(1e-12, sort(abs.(real(evals)))[end:-1:1])

plot(1:length(svals), real(svals)/real(svals[1]), label="svd", yscale=:log10)
plot!(1:length(revevals), real(revevals)/real(revevals[1]), label="ev",yscale=:log10)

In [ ]:
x_vecs1d = [randn()/8 for _ in 1:2000]
truth_mat1d  = mat_kern.(x_vecs1d, permutedims(x_vecs1d))

svecs1d, svals1d = svd(truth_mat1d);
scatter(x_vecs1d, svecs1d[:, 1])
scatter!(x_vecs1d, svecs1d[:, 2])
scatter!(x_vecs1d, svecs1d[:, 3])
scatter!(x_vecs1d, svecs1d[:, 4], xlim=(-b/2,b/2), ylim=(-0.05,0.05))
# scatter!(x_vecs2d, svecs2d[:, 5], xlim=(-b/2,b/2), ylim=(-0.05,0.05))
# scatter!(x_vecs2d, svecs2d[:, 6])


In [ ]:
othermat_kern(x,y) =exp(-norm(x-y)^2)

othertruth_mat1d  = othermat_kern.(x_vecs1d, permutedims(x_vecs1d))

othersvecs1d, othersvals1d = svd(othertruth_mat1d);
scatter(x_vecs1d, othersvecs1d[:, 1])
scatter!(x_vecs1d, othersvecs1d[:, 2])
scatter!(x_vecs1d, othersvecs1d[:, 3])
scatter!(x_vecs1d, othersvecs1d[:, 4], xlim=(-b/2,b/2), ylim=(-0.05,0.05))
# scatter!(x_vecs2d, svecs2d[:, 5], xlim=(-b/2,b/2), ylim=(-0.05,0.05))
# scatter!(x_vecs2d, svecs2d[:, 6])


In [ ]:

for i in 1:length(x_vecs1d)
    if abs(othersvecs1d[i, 3]) < 1e-4
        println(x_vecs1d[i])
    end
end
scatter(x_vecs1d, othersvecs1d[:, 3], label="true")

scatter!(x_vecs1d, 
    [1.1000114xv^2-0.016861549xv-0.015763104 for xv in x_vecs1d] , 
    ) #xlim = (-0.03,0.03),
#     #ylim = (-0.03,0))



In [ ]:
x_vecs2d = [randn(2)/8 for _ in 1:2000]
truth_mat2d  = mat_kern.(x_vecs2d, permutedims(x_vecs2d))

svecs2d, svals2d = svd(truth_mat2d);
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 1])


In [ ]:
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 2], camera=(80,0))


In [ ]:
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 3])


In [ ]:
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 4])


In [ ]:
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 5], camera=(0,0))


In [ ]:
scatter([xv[1] for xv in x_vecs2d],[xv[2] for xv in x_vecs2d] , svecs2d[:, 6], camera=(90,0))


In [ ]:
evecs = all_evecs[end]

In [ ]:
evalmag_to_evec = Dict()
for i in 1:length(evals)
    evalmag_to_evec[abs(evals[i])] = evecs[:,i]
end
sorted_evecs = sort(collect(evalmag_to_evec), by = x->x[1])[end:-1:1];

In [ ]:
start_i = 1
end_i = 5
plt = plot(1:size(evecs,1), real(sorted_evecs[1][2]))
for i in start_i+1:end_i
    plt = plot!(1:size(evecs,1), real(sorted_evecs[i][2]), xlim=(0,200), legend=false)
end
plt

In [ ]:
sorted_evecs[2][2][37:50]

In [ ]:
sorted_evecs[1][2][1:14]

In [ ]:
pole_count = 0
for j in 0:dover2
    for h in 1:length(get_multiindices(d, j))
        for new_i in 0:(fkt_deg - j)
            pole_count += 1
            println("entry ",pole_count, " is ", j,",", h,",", new_i)
        end
    end
end

In [ ]:


generate the same singular function plots except using eigenfunctions from cheb expansion
    (2d wrangling)